In [5]:
from os import listdir
import pandas as pd
import numpy as np
import csv
import glob
import tensorly as tl

In [6]:
fdata=pd.read_csv('imputed_data.csv') #Data imputed_data.csv prepared at matlab (1552210,41) with foward-fill imputation.


In [7]:
lengths=pd.read_csv('lengths.csv',header=None) # Individual Length of Stay (LOS) fo all 40336 patients from taining data

In [8]:
#Re-arranging imputed 1552210 hours of data in 40336 recods
k=0;
b=[]
intr=lengths.iloc[:].values
intr1=np.array(intr,dtype=None)
x=range(0,40336)
for n in x:
    var=[]    
    aa=intr1[n]
    a2=aa[0]
    var=pd.DataFrame(fdata.iloc[k:k+a2,:].values)
    b.append(var)
    k=k+a2

In [9]:
#compute correlation of every 6 hours of data for 24 covariates(excluding covariates with 
#more than 95 % of missing values(10 covariates) and 6 demographics)
# This cell corresponds to 80% of training data ~32222 patient records equivalent tp 1241759 hours of length of stay 
#out of 40336 patients. So this forms the training data for first fold

from tensorly.decomposition import tucker
from tensorly.decomposition import parafac
from tensorly.cp_tensor import cp_to_tensor, CPTensor
feat2D = np.zeros((100,1241759))
varr=0
for rec in range(32222):
    A=b[rec]
    A2=A.drop(columns =[7 ,8 ,13, 14 ,16, 20, 22, 26, 27, 32 ,34 ,35,36 ,37 ,38, 39 ])

    k=6
    u=len(A2)
    feat=[]
    for j in range(u-5):  # Start appending matrices from 7th hour
        var1=[]
        corrd=[]
        var1=A2.iloc[j:k,:]
        corrd=var1.corr()       #6 x 24 ----> 24 x 24
        corrd.fillna(0, inplace=True)
    
        k=k+1
        feat.append(corrd)
    
    var2=feat[0]        #replicate the 7th frame (matrix) to first 6 frames
    aa=[]
    for k in range(5):
        aa.append(var2)
    
    newfeat=aa+feat 
    unfld=[]
    for ll in range(len(newfeat)):
        v1=np.array(newfeat[ll])
        v2=v1.reshape(576)           # 24x24 =576
        unfld.append(v2)# shape is 54,576

    npunfld=np.array(unfld).T #576,54
    npunfld1=npunfld.reshape(24,24,len(newfeat))

    #tensor formation
    tensor = tl.tensor(npunfld1, dtype=tl.float64)
    unfolded = tl.unfold(tensor, mode=0)
    tl.fold(unfolded, mode=0, shape=tensor.shape)

    # Apply Tucker decomposition
    Tucker_tensor = tucker(tensor,[10,10,u]) # R1=10, R2=10, R3= length of stay.
    coreT=Tucker_tensor[0]

    fm=coreT.reshape(100,len(A2)) # reshape core for 100 features...!
    feat2D[:,varr:varr+len(A2)]=fm
    #print(rec)
    varr=varr+len(A2)


C:\Users\Naimahmed\anaconda3\lib\site-packages\tensorly\backend\core.py:779: RuntimeWarning: invalid value encountered in sqrt
  S = np.where(np.abs(S) <= np.finfo(S.dtype).eps, 0, np.sqrt(S))
C:\Users\Naimahmed\anaconda3\lib\site-packages\tensorly\backend\core.py:780: RuntimeWarning: invalid value encountered in less_equal
  V = np.dot(matrix.T.conj(), U * np.where(np.abs(S) <= np.finfo(S.dtype).eps, 0, 1/S)[None, :])
C:\Users\Naimahmed\anaconda3\lib\site-packages\tensorly\backend\core.py:780: RuntimeWarning: divide by zero encountered in true_divide
  V = np.dot(matrix.T.conj(), U * np.where(np.abs(S) <= np.finfo(S.dtype).eps, 0, 1/S)[None, :])


In [10]:
feat2DT=feat2D.T
feat2DT.shape

(1241759, 100)

In [11]:
#compute correlation of every 6 hours of data for 24 covariates(excluding covariates with 
#more than 59 % of missing values(10) and 6 demographics)

from tensorly.decomposition import tucker
from tensorly.decomposition import parafac
from tensorly.cp_tensor import cp_to_tensor, CPTensor
feat2Dtest = np.zeros((100,310451))
varr=0
for rec in range(32222,40336):
    A=b[rec]
    A2=A.drop(columns =[7 ,8 ,13, 14 ,16, 20, 22, 26, 27, 32 ,34 ,35,36 ,37 ,38, 39])

    k=6
    u=len(A2)
    feat=[]
    for j in range(u-5):  # Start appending matrices from 7th hour
        var1=[]
        corrd=[]
        var1=A2.iloc[j:k,:]
        corrd=var1.corr()       #6 x 24 ----> 24 x 24
        corrd.fillna(0, inplace=True)
    
        k=k+1
        feat.append(corrd)
    
    var2=feat[0]        #replicate the 7th frame (matrix) to first 6 frames
    aa=[]
    for k in range(5):
        aa.append(var2)
    
    newfeat=aa+feat 
    unfld=[]
    for ll in range(len(newfeat)):
        v1=np.array(newfeat[ll])
        v2=v1.reshape(576)           # 24x24 =576
        unfld.append(v2)# shape is 54,576

    npunfld=np.array(unfld).T #576,54
    npunfld1=npunfld.reshape(24,24,len(newfeat))

    #tensor formation
    tensor_test = tl.tensor(npunfld1, dtype=tl.float64)
    unfolded = tl.unfold(tensor_test, mode=0)
    tl.fold(unfolded, mode=0, shape=tensor_test.shape)

    # Apply Tucker decomposition
    Tucker_tensor_test = tucker(tensor_test,[10,10,u]) # R1=10, R2=10, R3= length of stay.
    coreT_test=Tucker_tensor_test[0]

    fm_test=coreT_test.reshape(100,len(A2)) # rehape core for 100 features...!
    feat2Dtest[:,varr:varr+len(A2)]=fm_test
    #print(rec)
    varr=varr+len(A2)


In [12]:
sepsis_data1 = pd.read_csv('labels_sir.csv')
Y=sepsis_data1.values
Y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [ ]:
traindata1=feat2DT
traindata1.shape
trainlabels=Y[0:1241759];
trainlabels.shape

In [14]:
testdata1=feat2Dtest.T
testdata1.shape
testlabels=Y[1241759:1552210];
testlabels.shape

(310451, 1)

In [15]:
dummy = pd.read_csv('range1.csv')
dummy

,range1,range2
0,1,54
1,55,77
2,78,125
3,126,154
4,155,202
...,...,...
40331,1552034,1552081
40332,1552082,1552106
40333,1552107,1552155
40334,1552156,1552175


In [16]:
#range_test=range(32223:end,:)-1241759;
range_test=dummy.iloc[32222:40336,:].values
range_test1=range_test-1241759
range_test1

array([[     1,     37],
       [    38,     83],
       [    84,    134],
       ...,
       [310348, 310396],
       [310397, 310416],
       [310417, 310451]], dtype=int64)

In [17]:
#range_test=range(32223:end,:)-1241759;
range_test=dummy[32222:40336]
range_test1=range_test-1241759
a=range_test1.iloc[:,1]
a
#range_test2=range_test1[:,0:1]

32222        37
32223        83
32224       134
32225       142
32226       150
          ...  
40331    310322
40332    310347
40333    310396
40334    310416
40335    310451
Name: range2, Length: 8114, dtype: int64

In [18]:
b=[]
x=range(0,8114)
for n in x:
      b.append(a[n+32222])

In [19]:
import numpy as np, os, os.path, sys, warnings

def evaluate_sepsis_score(label_directory, prediction_directory,num_files):
    # Set parameters.
    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0

    
    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        labels = cohort_labels[k]
        num_rows          = len(labels)
        observed_predictions = cohort_predictions[k]
        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels):
            t_sepsis = np.argmax(labels) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)

    return  normalized_observed_utility


In [20]:
def compute_auc(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not 0 <= prediction <= 1:
                warnings.warn('Predictions do not satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]

    n = len(labels)
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort the predicted probabilities from largest to
    # smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels)
            tn[j] = n - fn[j]
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest predicted probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function with TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant with TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc

# The compute_accuracy_f_measure function computes the accuracy and F-measure
# for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'accuracy' is a scalar that gives the accuracy of the predictions using its
#   binarized predictions.
#
#   'f_measure' is a scalar that gives the F-measure of the predictions using its
#   binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)
#   In [4]: accuracy
#   Out[4]: 0.666666666667
#   In [5]: f_measure
#   Out[5]: 0.666666666667

def compute_accuracy_f_measure(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    n = len(labels)
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif not labels[i] and predictions[i]:
            fp += 1
        elif labels[i] and not predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

# The compute_prediction_utility function computes the total time-dependent
# utility for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'utility' is a scalar that gives the total time-dependent utility of the
#   algorithm using its binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: utility = compute_prediction_utility(labels, predictions)
#   In [4]: utility
#   Out[4]: 3.388888888888889

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

In [21]:
cohort_labels        = []
cohort_predictions   = []
cohort_probabilities = []

In [26]:
space = {
            #'max_features':  hp.choice('max_features',['sqrt','log2', 0.5,0.75]), 
            'num_leaves': (40, 100), 
            'reg_lambda': (0, 10),
            'lambda_l1': (0, 5),
            'lambda_l2': (5, 10),
            'max_depth': (10,20),
            'scale_pos_weight':(18,40),
            'learning_rate':(0.05,0.2),
            'min_data_in_leaf':(40,100),
            'reg_alpha': (0, 10),
            #'min_split_gain':(0.01,10)
    
            
         }

In [27]:
def objective(num_leaves,
             reg_lambda,
             lambda_l1,
             lambda_l2,
             max_depth,
             scale_pos_weight,
             learning_rate,
             min_data_in_leaf,
             reg_alpha):
             #min_split_gain):
        num_leaves = int(num_leaves)
        min_data_in_leaf = int(min_data_in_leaf)
        max_depth = int(max_depth)
        #n_estimators =int(n_estimators)
        params = {
           # 'max_features': space['max_features'],
            'num_leaves':num_leaves,
            'reg_lambda':reg_lambda,
            'lambda_l1':lambda_l1,
            'lambda_l2':lambda_l2,
            'max_depth':max_depth,
            'scale_pos_weight':scale_pos_weight,
            'learning_rate':learning_rate,
            'min_data_in_leaf': min_data_in_leaf,
            'reg_alpha': reg_alpha
            #'min_split_gain':min_split_gain
             }
        #X_train, X_test, y_train, y_test = train_test_split(traindata1, trainlabels, test_size=0.2, random_state=42)
        import lightgbm as lgb
        clf = lgb.LGBMClassifier(n_estimators=100, is_unbalanced=True, **params)
        clf.fit(traindata1, trainlabels)#,early_stopping_rounds=100, eval_metric="f1_score",
        #eval_set=[(traindata1, trainlabels), (testdata1, testlabels)])
        pred_scores = clf.predict_proba(testdata1)
        pred_scores2=pred_scores[:,1]
        y_pred=clf.predict(testdata1)
        n=0
        m=0
        for k in b:
            cohort_labels.append(testlabels[m:k])
            cohort_predictions.append(y_pred[m:k])
            cohort_probabilities.append(pred_scores2[m:k])
            m=b[n]
            n=n+1
        
        normalized_observed_utility = evaluate_sepsis_score(cohort_labels,cohort_predictions,len(cohort_labels));
        return normalized_observed_utility

In [ ]:
#from bayes_opt import BayesianOptimization
LGB_BO = BayesianOptimization(objective, space)
print(LGB_BO.space.keys)
import warnings
init_points = 16
n_iter = 16
with warnings.catch_warnings():  
    warnings.filterwarnings('ignore')    
LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

In [81]:
import lightgbm as lgb

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
#model = XGBClassifier(learning_rate=0.01, n_estimators=100, subsample=0.8, max_depth=3,objective='binary:logistic', reg_alpha=0.3,colsample_bytree=0.4,gamma=10,scale_pos_weight= 20)
#model.fit(traindata1, trainlabels)

#model = lgb.LGBMClassifier(num_leaves=60,min_data_in_leaf= 120,max_depth= 1,learning_rate= 0.01,reg_alpha= 0, reg_lambda= 0,metric= 'f1_score', scale_pos_weight= 20)
model = lgb.LGBMClassifier(lambda_l1= 4.044,lambda_l2= 6.72,learning_rate= 0.08874, max_depth =17,min_data_in_leaf=52,num_leaves=45,reg_alpha=9.005,reg_lambda= 6.675,scale_pos_weight=23.13)
model.fit(traindata1, trainlabels)
#1.938    |  7.643    |  0.08446  |  15.6     |  47.77    |  55.42    |  1.944    |  3.547    |  27.67    |              U36.75
#0.363    |  1.083    |  8.687    |  0.05214  |  14.06    |  65.66    |  65.88    |  8.095    |  4.26     |  24.9     |  U37.2
#0.3684   |  4.044    |  6.72     |  0.08874  |  16.15    |  51.02    |  44.16    |  9.005    |  6.675    |  23.13    |  U37.66


C:\Users\Naimahmed\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Naimahmed\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', lambda_l1=4.044, lambda_l2=6.72,
               learning_rate=0.08874, max_depth=17, min_child_samples=20,
               min_child_weight=0.001, min_data_in_leaf=52, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=45, objective=None,
               random_state=None, reg_alpha=9.005, reg_lambda=6.675,
               scale_pos_weight=23.13, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [82]:
pred_scores = model.predict_proba(testdata1)
pred_scores

array([[0.9981466 , 0.0018534 ],
       [0.97989952, 0.02010048],
       [0.99393019, 0.00606981],
       ...,
       [0.54618489, 0.45381511],
       [0.48391564, 0.51608436],
       [0.37977986, 0.62022014]])

In [83]:
pred_scores2=pred_scores[:,1]
pred_scores2.shape

(310451,)

In [84]:
y_pred=model.predict(testdata1)
y_pred.shape

(310451,)

In [85]:
cohort_labels        = []
cohort_predictions   = []
cohort_probabilities = []

In [86]:
n=0
m=0
for k in b:
    cohort_labels.append(testlabels[m:k])
    cohort_predictions.append(y_pred[m:k])
    cohort_probabilities.append(pred_scores2[m:k])
    m=b[n]
    n=n+1

In [87]:
normalized_observed_utility = evaluate_sepsis_score(cohort_labels,cohort_predictions,len(cohort_labels));
normalized_observed_utility

0.3766669128508125